In [59]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
#import user defined function
#udf , stringtype(for split)
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from range_key_dict import RangeKeyDict
# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("local") \
        .appName("B.1")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext


In [65]:
#df = spark_session.read.format("csv").option("header", "true").load("download-data?year=2017")
SICdf = spark_session.read.csv('UK2017_SIC.csv', header="true", inferSchema="true", multiLine="true", escape="\"")
SICdf.createOrReplaceTempView("SIC")
PayGdf = spark_session.read.csv('download-data?year=2017', header="true", inferSchema="true", multiLine="true", escape="\"")
PayGdf.createOrReplaceTempView("PayGap")
SICdf.printSchema()


root
 |-- abbr: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Min: integer (nullable = true)
 |-- Max: integer (nullable = true)



In [66]:
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import string

def first_two(SicCodes):
    if SicCodes is None:
        return None
    else:
        FirstCode = (SicCodes.split(','))[0]#.substr(1,2)
        if FirstCode == '1':
            return '00'
        else:
            if len(FirstCode) <= 4:
                return ('0' + FirstCode[:1])
            else:
                return FirstCode[:2]
        
udf_first_two =udf(first_two, StringType())

Splited_PayGdf = PayGdf.withColumn("SicCodesSplit", udf_first_two("SicCodes"))
#Splited_PayGdf.select('SicCodesSplit').take(30)
Splited_PayGdf=Splited_PayGdf.withColumn("SicCodesSplit", Splited_PayGdf["SicCodesSplit"].cast(IntegerType()))
Splited_PayGdf.printSchema()

root
 |-- EmployerName: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- CompanyNumber: string (nullable = true)
 |-- SicCodes: string (nullable = true)
 |-- DiffMeanHourlyPercent: double (nullable = true)
 |-- DiffMedianHourlyPercent: double (nullable = true)
 |-- DiffMeanBonusPercent: double (nullable = true)
 |-- DiffMedianBonusPercent: double (nullable = true)
 |-- MaleBonusPercent: double (nullable = true)
 |-- FemaleBonusPercent: double (nullable = true)
 |-- MaleLowerQuartile: double (nullable = true)
 |-- FemaleLowerQuartile: double (nullable = true)
 |-- MaleLowerMiddleQuartile: double (nullable = true)
 |-- FemaleLowerMiddleQuartile: double (nullable = true)
 |-- MaleUpperMiddleQuartile: double (nullable = true)
 |-- FemaleUpperMiddleQuartile: double (nullable = true)
 |-- MaleTopQuartile: double (nullable = true)
 |-- FemaleTopQuartile: double (nullable = true)
 |-- CompanyLinkToGPGInfo: string (nullable = true)
 |-- ResponsiblePerson: string (nullable = 

In [72]:

#abbList = abbList.map(lambda x: ((x[1], x[2]), x[0]))
#abbList.take(30)
add_PayGdf = Splited_PayGdf.join(SICdf,(Splited_PayGdf.SicCodesSplit>=SICdf.Min)&(Splited_PayGdf.SicCodesSplit<=SICdf.Max))
add_PayGdf.select('Sector', 'EmployerName', 'DiffMeanHourlyPercent').show()
Sort_add_PayGdf = add_PayGdf.groupBy('Sector').avg('DiffMeanHourlyPercent')
#add_PayGdf.sort(add_PayGdf.avg(DiffMeanHourlyPercent).desc()).collect()


+--------------------+--------------------+---------------------+
|              Sector|        EmployerName|DiffMeanHourlyPercent|
+--------------------+--------------------+---------------------+
|           EDUCATION|"Bryanston School...|                 18.0|
|WHOLESALE AND RET...|"RED BAND" CHEMIC...|                  2.3|
|INFORMATION AND C...|         118 LIMITED|                  1.7|
|ADMINISTRATIVE AN...|   123 EMPLOYEES LTD|                 41.0|
|ARTS ENTERTAINMEN...|        1610 LIMITED|                -22.0|
|ACCOMMODATION AND...|1879 EVENTS MANAG...|                 13.4|
|ARTS ENTERTAINMEN...|1LIFE MANAGEMENT ...|                 15.1|
|HUMAN HEALTH AND ...|  1ST HOME CARE LTD.|                 15.0|
|ACCOMMODATION AND...| 1STOP HALAL LIMITED|                 11.9|
|AGRICULTURE FORES...|2 AGRICULTURE LIM...|                 13.4|
|       MANUFACTURING|2 SISTERS FOOD GR...|                 15.3|
|       MANUFACTURING|2 SISTERS POULTRY...|                  4.9|
|       MA

In [73]:
Sort_add_PayGdf.printSchema()
Sort_add_PayGdf.sort('avg(DiffMeanHourlyPercent)', ascending = False).show()

root
 |-- Sector: string (nullable = true)
 |-- avg(DiffMeanHourlyPercent): double (nullable = true)

+--------------------+--------------------------+
|              Sector|avg(DiffMeanHourlyPercent)|
+--------------------+--------------------------+
|FINANCIAL AND INS...|        26.421882951653938|
|        CONSTRUCTION|        21.707570977917985|
|MINING AND QUARRYING|        20.729999999999997|
|ARTS ENTERTAINMEN...|        20.689416058394166|
|INFORMATION AND C...|        19.500473933649314|
|PROFESSIONAL SCIE...|        18.477335164835157|
|REAL ESTATE ACTIV...|        16.024799999999995|
|ELECTRICITY GAS S...|         15.27818181818182|
|WHOLESALE AND RET...|         14.92169999999999|
|       MANUFACTURING|        13.301317715959009|
|AGRICULTURE FORES...|                      12.8|
|OTHER SERVICE ACT...|         12.46292682926829|
|           EDUCATION|        11.714520547945208|
|ADMINISTRATIVE AN...|        11.274045801526723|
|TRANSPORTATION AN...|        10.213910761154864

In [43]:
range_key_dict = {
        zip(range(1, 3), 'A'),}
range_key_dict[3]

TypeError: 'set' object does not support indexing

In [39]:
#NewPayGdf=NewPayGdf.withColumn("NewSIC", NewPayGdf["NewSIC"].cast(IntegerType()))
#SICdf=SICdf.withColumn("Max", SICdf["Max"].cast(IntegerType()))
#SICdf.printSchema()
#SICdf.show()
SICdf.select('abbr', 'Min', 'Max').rdd.map(tuple).take(5)
#building range dictionary:
range_key_dict = {
        tuple(range(1, 3)): 'A',
        tuple(range(5, 9)): 'B',
        tuple(range(10, 33)): 'C',
        tuple(range(35, 35)): 'D',
        tuple(range(36, 39)): 'E',
        tuple(range(41, 43)): 'F',
        tuple(range(45, 47)): 'G',
        tuple(range(49, 53)): 'H',
        tuple(range(55, 56)): 'I',
        tuple(range(58, 63)): 'J',
        tuple(range(64, 66)): 'K',
        tuple(range(68, 68)): 'L',
        tuple(range(69, 75)): 'M',
        tuple(range(77, 82)): 'N',
        tuple(range(84, 84)): 'O',
        tuple(range(85, 85)): 'P',
        tuple(range(86, 88)): 'Q',
        tuple(range(90, 93)): 'R',
        tuple(range(94, 96)): 'S',
        tuple(range(97, 98)): 'T',
        tuple(range(99, 99)): 'U',
        
    }

range_key_dict(5)


TypeError: 'dict' object is not callable

In [9]:
PayGdf.printSchema()

root
 |-- EmployerName: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- CompanyNumber: string (nullable = true)
 |-- SicCodes: string (nullable = true)
 |-- DiffMeanHourlyPercent: double (nullable = true)
 |-- DiffMedianHourlyPercent: double (nullable = true)
 |-- DiffMeanBonusPercent: double (nullable = true)
 |-- DiffMedianBonusPercent: double (nullable = true)
 |-- MaleBonusPercent: double (nullable = true)
 |-- FemaleBonusPercent: double (nullable = true)
 |-- MaleLowerQuartile: double (nullable = true)
 |-- FemaleLowerQuartile: double (nullable = true)
 |-- MaleLowerMiddleQuartile: double (nullable = true)
 |-- FemaleLowerMiddleQuartile: double (nullable = true)
 |-- MaleUpperMiddleQuartile: double (nullable = true)
 |-- FemaleUpperMiddleQuartile: double (nullable = true)
 |-- MaleTopQuartile: double (nullable = true)
 |-- FemaleTopQuartile: double (nullable = true)
 |-- CompanyLinkToGPGInfo: string (nullable = true)
 |-- ResponsiblePerson: string (nullable = 

In [34]:
from range_key_dict import RangeKeyDict

if __name__ == '__main__':
    range_key_dict = RangeKeyDict({
        (0, 90): 'A',
        (100, 200): 'B',
        (201,204): 'C',
    })
assert range_key_dict[200] == 'B'

KeyError: 200

In [49]:
PayGdf.select('SicCodes').take(20)


[Row(SicCodes='85310'),
 Row(SicCodes='47730'),
 Row(SicCodes='61900'),
 Row(SicCodes='78300'),
 Row(SicCodes='93110'),
 Row(SicCodes='56210,\r\n70229'),
 Row(SicCodes='93110,\r\n93130,\r\n93290'),
 Row(SicCodes='86900,\r\n88100'),
 Row(SicCodes='56290'),
 Row(SicCodes='1470,\r\n10910'),
 Row(SicCodes='10120'),
 Row(SicCodes='10120'),
 Row(SicCodes='10110'),
 Row(SicCodes='82200'),
 Row(SicCodes='56103'),
 Row(SicCodes='52290'),
 Row(SicCodes='78200'),
 Row(SicCodes='1,\r\n86210'),
 Row(SicCodes='81210'),
 Row(SicCodes='52290')]

In [59]:
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import string

def first_two(SicCodes):
    if SicCodes is None:
        return None
    else:
        FirstCode = (SicCodes.split(','))[0]#.substr(1,2)
        if FirstCode == '1':
            return '00'
        else:
            if len(FirstCode) <= 4:
                return ('0' + FirstCode[:1])
            else:
                return FirstCode[:2]
        
udf_first_two =udf(first_two, StringType())

Splited_PayGdf = PayGdf.withColumn("SicCodesSplit", udf_first_two("SicCodes"))
Splited_PayGdf.select('SicCodesSplit').take(30)

[Row(SicCodesSplit='85'),
 Row(SicCodesSplit='47'),
 Row(SicCodesSplit='61'),
 Row(SicCodesSplit='78'),
 Row(SicCodesSplit='93'),
 Row(SicCodesSplit='56'),
 Row(SicCodesSplit='93'),
 Row(SicCodesSplit='86'),
 Row(SicCodesSplit='56'),
 Row(SicCodesSplit='01'),
 Row(SicCodesSplit='10'),
 Row(SicCodesSplit='10'),
 Row(SicCodesSplit='10'),
 Row(SicCodesSplit='82'),
 Row(SicCodesSplit='56'),
 Row(SicCodesSplit='52'),
 Row(SicCodesSplit='78'),
 Row(SicCodesSplit='00'),
 Row(SicCodesSplit='81'),
 Row(SicCodesSplit='52'),
 Row(SicCodesSplit='20'),
 Row(SicCodesSplit='82'),
 Row(SicCodesSplit='70'),
 Row(SicCodesSplit='93'),
 Row(SicCodesSplit='41'),
 Row(SicCodesSplit='70'),
 Row(SicCodesSplit='56'),
 Row(SicCodesSplit='33'),
 Row(SicCodesSplit='56'),
 Row(SicCodesSplit='68')]

str

In [35]:
numdf = PayGdf.select(PayGdf.SicCodes.substr(1,2))
#PayGdf.createOrReplaceTempView("PayGap")
#sqlDF = spark_session.sql("SELECT SicCodes FROM PayGap")


In [95]:
numdf.show()

+-------------------------+
|substring(SicCodes, 1, 2)|
+-------------------------+
|                       85|
|                       47|
|                       61|
|                       78|
|                       93|
|                       56|
|                       93|
|                       86|
|                       56|
|                       14|
|                       10|
|                       10|
|                       10|
|                       82|
|                       56|
|                       52|
|                       78|
|                       1,|
|                       81|
|                       52|
+-------------------------+
only showing top 20 rows



In [101]:
NewPayGdf = PayGdf.withColumn('NewSIC', PayGdf.SicCodes.substr(1,2))


In [114]:
NewPayGdf = NewPayGdf.filter(NewPayGdf['NewSIC']!='1,') #.select('NewSIC').show()
test = NewPayGdf.na.replace('85','100','NewSIC')
#NewPayGdf.select("NewSIC").show()
test.select('NewSIC').show()

+------+
|NewSIC|
+------+
|   100|
|    47|
|    61|
|    78|
|    93|
|    56|
|    93|
|    86|
|    56|
|    14|
|    10|
|    10|
|    10|
|    82|
|    56|
|    52|
|    78|
|    81|
|    52|
|    20|
+------+
only showing top 20 rows



In [89]:
NewPayGdf=NewPayGdf.withColumn("NewSIC", NewPayGdf["NewSIC"].cast(IntegerType()))
NewPayGdf.printSchema()

root
 |-- EmployerName: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- CompanyNumber: string (nullable = true)
 |-- SicCodes: string (nullable = true)
 |-- DiffMeanHourlyPercent: double (nullable = true)
 |-- DiffMedianHourlyPercent: double (nullable = true)
 |-- DiffMeanBonusPercent: double (nullable = true)
 |-- DiffMedianBonusPercent: double (nullable = true)
 |-- MaleBonusPercent: double (nullable = true)
 |-- FemaleBonusPercent: double (nullable = true)
 |-- MaleLowerQuartile: double (nullable = true)
 |-- FemaleLowerQuartile: double (nullable = true)
 |-- MaleLowerMiddleQuartile: double (nullable = true)
 |-- FemaleLowerMiddleQuartile: double (nullable = true)
 |-- MaleUpperMiddleQuartile: double (nullable = true)
 |-- FemaleUpperMiddleQuartile: double (nullable = true)
 |-- MaleTopQuartile: double (nullable = true)
 |-- FemaleTopQuartile: double (nullable = true)
 |-- CompanyLinkToGPGInfo: string (nullable = true)
 |-- ResponsiblePerson: string (nullable = 

In [134]:
abbList = SICdf.select('abbr','Min','Max').rdd.map(tuple)
abbList = abbList.map(lambda x: (x[0], (x[1], x[2])))
abbList.take(5)
#abbList = abbList.map(lambda x: (x[0]), (x[1], x[2]))

[('A', (1, 3)),
 ('B', (5, 9)),
 ('C', (10, 33)),
 ('D', (35, 35)),
 ('E', (36, 39))]

In [106]:
rdd.take(5)

[Row(abbr='A', Sector='AGRICULTURE FORESTRY AND FISHING', Min=1, Max=3),
 Row(abbr='B', Sector='MINING AND QUARRYING', Min=5, Max=9),
 Row(abbr='C', Sector='MANUFACTURING', Min=10, Max=33),
 Row(abbr='D', Sector='ELECTRICITY GAS STEAM AND AIR CONDITIONING SUPPLY', Min=35, Max=35),
 Row(abbr='E', Sector='WATER SUPPLY SEWERAGE WASTE MANAGEMENT AND REMEDIATION ACTIVITIES', Min=36, Max=39)]